Complete the exercises below For **Assignment #13**.

Load the `ISLR2` and the `tidymodels` packages.

In [1]:
library('ISLR2')
library('tidymodels')

Warning message:
“Your system is mis-configured: ‘/var/db/timezone/localtime’ is not a symlink”
Warning message:
“‘/var/db/timezone/localtime’ is not identical to any known timezone file”
── Attaching packages ────────────────────────────────────── tidymodels 1.4.1 ──

✔ broom        1.0.9     ✔ recipes      1.3.1
✔ dials        1.4.2     ✔ rsample      1.3.1
✔ dplyr        1.1.4     ✔ tailor       0.1.0
✔ ggplot2      3.5.2     ✔ tidyr        1.3.1
✔ infer        1.0.9     ✔ tune         2.0.0
✔ modeldata    1.5.1     ✔ workflows    1.3.0
✔ parsnip      1.3.3     ✔ workflowsets 1.1.1
✔ purrr        1.1.0     ✔ yardstick    1.3.2

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()



In this assignment we will use the `Default` dataset which includes the default status for credit card customers (`default` variable) in addition to each customer's:

1. credit card balance (`balance` variable),
1. student status (`student` variable), and,
1. income (`income` variable).

In [2]:
Default |> head()

,default,student,balance,income
,<fct>,<fct>,<dbl>,<dbl>
1,No,No,729.5265,44361.625
2,No,Yes,817.1804,12106.135
3,No,No,1073.5492,31767.139
4,No,No,529.2506,35704.494
5,No,No,785.6559,38463.496
6,No,Yes,919.5885,7491.559


We will be modeling `default` with the customer features.

Before we begin let's count how many customers fall into each `default` category.

In [4]:
Default |> count(default)

default,n
<fct>,<int>
No,9667
Yes,333


The data is quite imbalanced. This will be important to keep in mind when we evaluate the performance of our model later. 

Run the code below to create and training data from `Default`. We will use the "test" dataset at the end to get a final evaluation of our best model's accuracy.

In [10]:
Default_split = initial_split(Default, prop = 0.90, strata = default)

Default_train = training(Default_split)
Default_test = testing(Default_split)

Create a logistic regression model called `mod`. Set the engine to `glm` and the mode to `classification`. 

In [8]:
mod = logistic_reg() |> set_engine('glm') |> set_mode('classification')

Our data is imbalanced. As such, a naive model that *always* predicts a customer to **not default** would be correct quite often. Let's start by calculating the "accuracy" of a naive model. This will be the baseline accuracy by which we evaluate other models.

In [11]:
# This code calculates the accuracy of a model that always predicts default to be "No"

Default_train |>
    mutate(.pred_naive = factor('No', levels = c('No', 'Yes'))) |>
    accuracy(truth = default, .pred_naive)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9668889


Let's use k-fold cross validation to evaluate the performance of a model where the outcome is `default` and the predictors are `income` and `balance`.

To start, use `vfold_cv` to generate 10 validation folds (i.e. set the `v` variable to 10). Set the `strata` argument to `default` so we preserve the distribution of `default` values in each fold.

Creat your folds below and use `glimpse` to look at the output table. Call your output folds tables "folds".

In [12]:
folds = vfold_cv(Default, v = 10, strata = default)
glimpse(folds)

Rows: 10
Columns: 2
$ splits <list> [<vfold_split[9000 x 1000 x 10000 x 4]>], [<vfold_split[9000 x…
$ id     <chr> "Fold01", "Fold02", "Fold03", "Fold04", "Fold05", "Fold06", "Fo…


The code below fits a model to each of your 10 folds. `collect_metrics` finds the average of evaluation metrics for each of your ten models. 

In [13]:
mod |> 
    fit_resamples(default ~ income + balance, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97380000,10,0.001860705,pre0_mod0_post0
brier_class,binary,0.02147219,10,0.001106647,pre0_mod0_post0
roc_auc,binary,0.94856996,10,0.003195302,pre0_mod0_post0


❓How does the model accuracy compare to the naive model from above?

The naive model had a mean accuracy of 0.967, compared to this model's accuracy of 0.974. This model's accuracy is higher by .7%, which is not much more.

Complete the cell below to evaluate a model also includes the `student` variable as as predictor.
1. use `default ~ income + balance + student` as the formula,
2. encode your `student` variable with `step_dummy`, and,
3. don't forget to `prep` your recipe!

In [17]:
rec = recipe(default ~ income + balance + student, data = Default) |> step_dummy(student)
# code threw an error that I wasn't supposed to prep the recipe, so I took that part out

mod |>
    fit_resamples(rec, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97330000,10,0.001789165,pre0_mod0_post0
brier_class,binary,0.02140501,10,0.001090131,pre0_mod0_post0
roc_auc,binary,0.94888444,10,0.003219556,pre0_mod0_post0


❓Does it appear that the model that includes `student` improves upon the first model with only `income` and `balance` as predictors?

The model with `student` has an accuracy of 0.9733 while the first model without it has an accuracy of 0.9738. This new model did not improve upon the first.

Finally, estimate the accuracy of an `default ~ income + balance` model on the test data, `Default_test`. 

❓Does our model outperform a naive model?

In [18]:
rec_test = recipe(default ~ income + balance + student, data = Default_test) |> step_dummy(student)

mod |>
    fit_resamples(rec_test, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97330000,10,0.001789165,pre0_mod0_post0
brier_class,binary,0.02140501,10,0.001090131,pre0_mod0_post0
roc_auc,binary,0.94888444,10,0.003219556,pre0_mod0_post0


The accuracy of this model on the test data has an accuracy of 0.9733 compared to the naive model's accuracy of 0.9667. There are minimal increases in accuracy but is more complex; therefore, this model does not necessarily outperform the naive model.